In [ ]:
# Cell 1: Imports and load processed data
import polars as pl
import numpy as np
from pathlib import Path
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

# Paths
DATA_DIR = Path("../data")
PROCESSED_DIR = DATA_DIR / "processed"

print("Loading processed data...")
all_events = pl.read_parquet(PROCESSED_DIR / "all_events_with_points.parquet")
matches = pl.read_parquet(PROCESSED_DIR / "matches_metadata.parquet")

# Load player ratings from previous approach (for position info)
player_ratings_old = pl.read_parquet(PROCESSED_DIR / "player_ratings_final.parquet")

print(f" Loaded {len(all_events):,} events")
print(f" Loaded {len(player_ratings_old)} players with positions")

ModuleNotFoundError: No module named 'sklearn'

In [ ]:
# Cell 2: Define role-specific metrics
print("DEFINING ROLE-SPECIFIC METRICS")

GRANULAR_METRICS = {
    
    'finishing': {
        'description': 'Ability to convert chances into goals',
        'events': ['SHOT'],
        'weights': {
            'GOAL': 10.0,
            'SAVED': 2.0,
            'OFF_TARGET': 0.3,
            'BLOCKED': 0.5,
        },
        'relevant_positions': ['forward', 'midfielder']
    },
    
    'chance_creation': {
        'description': 'Creating scoring opportunities for teammates',
        'events': ['PASS'],
        'weights': {
            'SHOT_ASSIST': 10.0,  # Assist
            'KEY_PASS': 5.0,      # Pass leading to shot (we'll calculate this)
            'INTO_PENALTY_AREA': 3.0,
        },
        'relevant_positions': ['forward', 'midfielder']
    },
    
    'ball_progression': {
        'description': 'Advancing the ball up the field',
        'events': ['PASS', 'CARRY'],
        'weights': {
            'PROGRESSIVE_PASS': 2.0,
            'PROGRESSIVE_CARRY': 2.5,
            'PASS_INTO_FINAL_THIRD': 1.5,
        },
        'relevant_positions': ['midfielder', 'defender']
    },
    
    'dribbling': {
        'description': 'Taking on opponents with the ball',
        'events': ['CARRY', 'DUEL'],
        'weights': {
            'PROGRESSIVE_CARRY': 1.5,
            'CARRY_INTO_BOX': 3.0,
            'DUEL_WON_ATTACKING': 1.0,
        },
        'relevant_positions': ['forward', 'midfielder']
    },
    

    'ball_winning': {
        'description': 'Winning possession from opponents',
        'events': ['DUEL', 'INTERCEPTION'],
        'weights': {
            'DUEL_WON_DEFENSIVE': 3.0,
            'DUEL_WON_MIDDLE': 2.0,
            'INTERCEPTION_DEFENSIVE': 2.5,
            'INTERCEPTION_MIDDLE': 2.0,
        },
        'relevant_positions': ['midfielder', 'defender']
    },
    
    'defensive_actions': {
        'description': 'Clearing danger and blocking attacks',
        'events': ['CLEARANCE', 'RECOVERY'],
        'weights': {
            'CLEARANCE_DEFENSIVE': 2.0,
            'CLEARANCE_MIDDLE': 1.0,
            'RECOVERY_DEFENSIVE': 1.5,
            'RECOVERY_MIDDLE': 1.0,
        },
        'relevant_positions': ['defender']
    },
    
    
    'passing_accuracy': {
        'description': 'Completing passes successfully',
        'events': ['PASS'],
        'weights': {
            'COMPLETE': 0.5,
            'INCOMPLETE': -0.3,
            'COMPLETE_UNDER_PRESSURE': 1.0,
        },
        'relevant_positions': ['midfielder', 'defender']
    },
    
    'long_passing': {
        'description': 'Successful long-range distribution',
        'events': ['PASS'],
        'weights': {
            'LONG_PASS_COMPLETE': 2.0,
            'LONG_PASS_INCOMPLETE': -0.5,
        },
        'relevant_positions': ['midfielder', 'defender']
    },
}

print(f" Defined {len(GRANULAR_METRICS)} granular metrics:")
for metric_name, metric_info in GRANULAR_METRICS.items():
    print(f"  • {metric_name:20s}: {metric_info['description']}")

DEFINING ROLE-SPECIFIC METRICS
✅ Defined 8 granular metrics:
  • finishing           : Ability to convert chances into goals
  • chance_creation     : Creating scoring opportunities for teammates
  • ball_progression    : Advancing the ball up the field
  • dribbling           : Taking on opponents with the ball
  • ball_winning        : Winning possession from opponents
  • defensive_actions   : Clearing danger and blocking attacks
  • passing_accuracy    : Completing passes successfully
  • long_passing        : Successful long-range distribution


In [ ]:
def is_progressive_pass(row):
    """Pass that advances ball >10m forward"""
    if row['end_coordinates_x'] is None or row['coordinates_x'] is None:
        return False
    return (row['end_coordinates_x'] - row['coordinates_x']) >= 10.0

def is_progressive_carry(row):
    """Carry that advances ball >10m forward"""
    if row['end_coordinates_x'] is None or row['coordinates_x'] is None:
        return False
    return (row['end_coordinates_x'] - row['coordinates_x']) >= 10.0

def is_long_pass(row):
    """Pass >30m in total distance"""
    if row['end_coordinates_x'] is None or row['coordinates_x'] is None:
        return False
    if row['end_coordinates_y'] is None or row['coordinates_y'] is None:
        return False
    distance = np.sqrt(
        (row['end_coordinates_x'] - row['coordinates_x'])**2 + 
        (row['end_coordinates_y'] - row['coordinates_y'])**2
    )
    return distance >= 30.0

def is_into_penalty_area(row):
    """Pass ending in penalty box"""
    if row['end_coordinates_x'] is None or row['end_coordinates_y'] is None:
        return False
    in_box_x = row['end_coordinates_x'] > 35.5
    in_box_y = -9.15 < row['end_coordinates_y'] < 9.15
    return in_box_x and in_box_y

def is_into_final_third(row):
    """Pass ending in attacking third"""
    if row['end_zone'] is None:
        return False
    return row['end_zone'] == 'attacking_third' and row['zone'] != 'attacking_third'

def is_carry_into_box(row):
    """Carry ending in penalty box"""
    return is_into_penalty_area(row)

print(" Helper functions defined")

✅ Helper functions defined


In [ ]:
# Cell 4: Calculate points for each granular metric

def calculate_finishing_points(row):
    """Points for finishing ability"""
    if row['event_type'] != 'SHOT':
        return 0.0
    
    result = row['result']
    if result == 'GOAL':
        return 10.0
    elif result == 'SAVED':
        return 2.0
    elif result == 'BLOCKED':
        return 0.5
    elif result == 'OFF_TARGET':
        return 0.3
    return 0.0

def calculate_chance_creation_points(row):
    """Points for creating chances"""
    if row['event_type'] != 'PASS':
        return 0.0
    
    pass_type = row.get('pass_type', None)
    
    # Assist
    if pass_type == 'SHOT_ASSIST':
        return 10.0
    
    # Pass into penalty area
    if row['result'] == 'COMPLETE' and is_into_penalty_area(row):
        return 3.0
    
    return 0.0

def calculate_ball_progression_points(row):
    """Points for advancing the ball"""
    points = 0.0
    
    # Progressive pass
    if row['event_type'] == 'PASS' and row['result'] == 'COMPLETE':
        if is_progressive_pass(row):
            points += 2.0
        elif is_into_final_third(row):
            points += 1.5
    
    # Progressive carry
    if row['event_type'] == 'CARRY' and row['result'] == 'COMPLETE':
        if is_progressive_carry(row):
            points += 2.5
    
    return points

def calculate_dribbling_points(row):
    """Points for dribbling/take-ons"""
    points = 0.0
    
    # Carry into box
    if row['event_type'] == 'CARRY' and row['result'] == 'COMPLETE':
        if is_carry_into_box(row):
            points += 3.0
        elif is_progressive_carry(row):
            points += 1.5
    
    # Duel won in attacking third (winning ball while dribbling)
    if row['event_type'] == 'DUEL' and row['result'] == 'WON':
        if row['zone'] == 'attacking_third':
            points += 1.0
    
    return points

def calculate_ball_winning_points(row):
    """Points for winning possession"""
    points = 0.0
    
    # Duels
    if row['event_type'] == 'DUEL' and row['result'] == 'WON':
        if row['zone'] == 'defensive_third':
            points += 3.0
        elif row['zone'] == 'middle_third':
            points += 2.0
    
    # Interceptions
    if row['event_type'] == 'INTERCEPTION':
        if row['zone'] == 'defensive_third':
            points += 2.5
        elif row['zone'] == 'middle_third':
            points += 2.0
        elif row['zone'] == 'attacking_third':
            points += 1.5
    
    return points

def calculate_defensive_actions_points(row):
    """Points for defensive clearances/recoveries"""
    points = 0.0
    
    # Clearances
    if row['event_type'] == 'CLEARANCE':
        if row['zone'] == 'defensive_third':
            points += 2.0
        elif row['zone'] == 'middle_third':
            points += 1.0
    
    # Recoveries
    if row['event_type'] == 'RECOVERY':
        if row['zone'] == 'defensive_third':
            points += 1.5
        elif row['zone'] == 'middle_third':
            points += 1.0
        elif row['zone'] == 'attacking_third':
            points += 0.8
    
    return points

def calculate_passing_accuracy_points(row):
    """Points for pass completion"""
    if row['event_type'] != 'PASS':
        return 0.0
    
    if row['result'] == 'COMPLETE':
        # Bonus if under pressure
        if row.get('is_under_pressure', False):
            return 1.0
        return 0.5
    elif row['result'] == 'INCOMPLETE':
        return -0.3
    
    return 0.0

def calculate_long_passing_points(row):
    """Points for long passing"""
    if row['event_type'] != 'PASS':
        return 0.0
    
    if is_long_pass(row):
        if row['result'] == 'COMPLETE':
            return 2.0
        elif row['result'] == 'INCOMPLETE':
            return -0.5
    
    return 0.0

print(" Metric calculation functions defined")
print("\nCalculating points for all events...")

import time
start_time = time.time()

# Convert to list for faster iteration
events_list = all_events.to_dicts()

# Calculate all metrics
finishing_points = []
chance_creation_points = []
ball_progression_points = []
dribbling_points = []
ball_winning_points = []
defensive_actions_points = []
passing_accuracy_points = []
long_passing_points = []

for i, row in enumerate(events_list):
    finishing_points.append(calculate_finishing_points(row))
    chance_creation_points.append(calculate_chance_creation_points(row))
    ball_progression_points.append(calculate_ball_progression_points(row))
    dribbling_points.append(calculate_dribbling_points(row))
    ball_winning_points.append(calculate_ball_winning_points(row))
    defensive_actions_points.append(calculate_defensive_actions_points(row))
    passing_accuracy_points.append(calculate_passing_accuracy_points(row))
    long_passing_points.append(calculate_long_passing_points(row))
    
    if (i + 1) % 200000 == 0:
        print(f"  Processed {i+1:,}/{len(events_list):,} events...")

# Add to dataframe
all_events = all_events.with_columns([
    pl.Series('finishing_points', finishing_points),
    pl.Series('chance_creation_points', chance_creation_points),
    pl.Series('ball_progression_points', ball_progression_points),
    pl.Series('dribbling_points', dribbling_points),
    pl.Series('ball_winning_points', ball_winning_points),
    pl.Series('defensive_actions_points', defensive_actions_points),
    pl.Series('passing_accuracy_points', passing_accuracy_points),
    pl.Series('long_passing_points', long_passing_points),
])

elapsed = time.time() - start_time
print(f"\n Calculated all metric points in {elapsed:.1f} seconds")

# Summary
print("GRANULAR METRICS SUMMARY")
metric_cols = ['finishing_points', 'chance_creation_points', 'ball_progression_points',
               'dribbling_points', 'ball_winning_points', 'defensive_actions_points',
               'passing_accuracy_points', 'long_passing_points']

for col in metric_cols:
    total = all_events[col].sum()
    positive = (all_events[col] > 0).sum()
    print(f"{col:30s}: Total={total:>8,.0f} | Positive events={positive:>7,}")

✅ Metric calculation functions defined

Calculating points for all events...
⏳ This will take ~30 seconds...
  Processed 200,000/962,990 events...
  Processed 400,000/962,990 events...
  Processed 600,000/962,990 events...
  Processed 800,000/962,990 events...

✅ Calculated all metric points in 12.9 seconds

GRANULAR METRICS SUMMARY
finishing_points              : Total=  14,965 | Positive events=  8,069
chance_creation_points        : Total=  34,328 | Positive events=  4,802
ball_progression_points       : Total= 159,290 | Positive events= 76,225
dribbling_points              : Total=  33,689 | Positive events= 21,656
ball_winning_points           : Total=  75,570 | Positive events= 33,010
defensive_actions_points      : Total=  75,067 | Positive events= 59,730
passing_accuracy_points       : Total= 164,197 | Positive events=237,831
long_passing_points           : Total=  29,475 | Positive events= 19,907


In [ ]:
# Cell 5: Aggregate granular metrics by player
print("Aggregating granular metrics by player...")

# Get position info from old ratings
position_info = player_ratings_old.select(['player_id', 'player_name', 'position', 
                                           'estimated_total_minutes', 'matches_played'])

# Aggregate metrics by player
player_granular = (
    all_events
    .filter(pl.col('player_id').is_not_null())
    .group_by(['player_id', 'team_id'])
    .agg([
        # Sum all granular metric points
        pl.sum('finishing_points').alias('total_finishing'),
        pl.sum('chance_creation_points').alias('total_chance_creation'),
        pl.sum('ball_progression_points').alias('total_ball_progression'),
        pl.sum('dribbling_points').alias('total_dribbling'),
        pl.sum('ball_winning_points').alias('total_ball_winning'),
        pl.sum('defensive_actions_points').alias('total_defensive_actions'),
        pl.sum('passing_accuracy_points').alias('total_passing_accuracy'),
        pl.sum('long_passing_points').alias('total_long_passing'),
        
        # Count events for diagnostics
        pl.len().alias('total_events'),
    ])
)

# Join with position info
player_granular = player_granular.join(position_info, on='player_id', how='left')

# Fill null player names with placeholder
player_granular = player_granular.with_columns([
    pl.when(pl.col('player_name').is_null())
    .then(pl.concat_str([pl.lit("Player_"), pl.col('player_id')]))
    .otherwise(pl.col('player_name'))
    .alias('player_name')
])

print(f" Fixed null player names")

# Filter for players with >500 minutes
player_granular = player_granular.filter(pl.col('estimated_total_minutes') >= 500)

print(f" Aggregated metrics for {len(player_granular)} players (>500 minutes)")

# Calculate per-90 for each metric
metric_columns = [
    'finishing', 'chance_creation', 'ball_progression', 'dribbling',
    'ball_winning', 'defensive_actions', 'passing_accuracy', 'long_passing'
]

for metric in metric_columns:
    player_granular = player_granular.with_columns([
        ((pl.col(f'total_{metric}') / pl.col('estimated_total_minutes')) * 90)
        .alias(f'{metric}_per90')
    ])

print("\n Calculated per-90 for all metrics")

# Show distribution
print("PER-90 METRIC DISTRIBUTIONS")


for metric in metric_columns:
    col = f'{metric}_per90'
    mean_val = player_granular[col].mean()
    max_val = player_granular[col].max()
    print(f"{metric:25s}: Mean={mean_val:>6.2f} | Max={max_val:>6.2f}")

# Top players by each metric
print("TOP 5 PLAYERS BY EACH METRIC (Per-90)")

for metric in metric_columns:
    print(f"\n{metric.upper().replace('_', ' ')}:")
    print(player_granular
          .select(['player_name', 'position', f'{metric}_per90', 'matches_played'])
          .sort(f'{metric}_per90', descending=True)
          .head(5))

Aggregating granular metrics by player...
✅ Fixed null player names
✅ Aggregated metrics for 421 players (>500 minutes)

✅ Calculated per-90 for all metrics

PER-90 METRIC DISTRIBUTIONS
finishing                : Mean=  1.45 | Max= 13.83
chance_creation          : Mean=  3.59 | Max= 45.12
ball_progression         : Mean= 16.38 | Max= 55.58
dribbling                : Mean=  3.39 | Max= 10.07
ball_winning             : Mean=  7.79 | Max= 25.69
defensive_actions        : Mean=  7.88 | Max= 24.38
passing_accuracy         : Mean= 16.74 | Max= 77.04
long_passing             : Mean=  3.13 | Max= 14.52

TOP 5 PLAYERS BY EACH METRIC (Per-90)

FINISHING:
shape: (5, 4)
┌─────────────────┬──────────┬─────────────────┬────────────────┐
│ player_name     ┆ position ┆ finishing_per90 ┆ matches_played │
│ ---             ┆ ---      ┆ ---             ┆ ---            │
│ str             ┆ str      ┆ f64             ┆ u32            │
╞═════════════════╪══════════╪═════════════════╪════════════════╡
│ H

In [22]:

# Check players with multiple positions
duplicate_players = (
    player_granular
    .group_by('player_id')
    .agg([
        pl.col('player_name').first(),
        pl.col('position').n_unique().alias('num_positions'),
        pl.col('position').unique().alias('positions'),
        pl.col('matches_played').sum().alias('total_matches'),
    ])
    .filter(pl.col('num_positions') > 1)
    .sort('num_positions', descending=True)
)

print(f"Players with multiple positions: {len(duplicate_players)}")
print("\nExamples:")
print(duplicate_players.head(10))

Players with multiple positions: 1

Examples:
shape: (1, 5)
┌───────────┬──────────────┬───────────────┬───────────────────────────┬───────────────┐
│ player_id ┆ player_name  ┆ num_positions ┆ positions                 ┆ total_matches │
│ ---       ┆ ---          ┆ ---           ┆ ---                       ┆ ---           │
│ str       ┆ str          ┆ u32           ┆ list[str]                 ┆ u32           │
╞═══════════╪══════════════╪═══════════════╪═══════════════════════════╪═══════════════╡
│ 1286      ┆ Nadiem Amiri ┆ 2             ┆ ["midfielder", "forward"] ┆ 92            │
└───────────┴──────────────┴───────────────┴───────────────────────────┴───────────────┘


In [ ]:
# CELL 5 - COMPLETELY REWRITTEN
# Cell 5: Aggregate by player (single position per player)
print("Aggregating granular metrics by player...")

# First, determine each player's primary position based on where they played most
player_positions = (
    all_events
    .filter(pl.col('player_id').is_not_null())
    .group_by('player_id')
    .agg([
        pl.mean('coordinates_x').alias('season_avg_x'),
        pl.len().alias('total_events'),
    ])
)

# Classify position based on season average
def classify_season_position(avg_x):
    if avg_x is None:
        return 'midfielder'
    if avg_x > 10:
        return 'forward'
    elif avg_x < -10:
        return 'defender'
    else:
        return 'midfielder'

positions = []
for row in player_positions.iter_rows(named=True):
    positions.append(classify_season_position(row['season_avg_x']))

player_positions = player_positions.with_columns([
    pl.Series('position', positions)
])

# Get player names and minutes from old ratings
player_info = player_ratings_old.select([
    'player_id', 'player_name', 'estimated_total_minutes', 'matches_played'
]).unique(subset=['player_id'])  # IMPORTANT: Take unique by player_id

# Aggregate metrics by player ONLY (not by team_id)
player_granular = (
    all_events
    .filter(pl.col('player_id').is_not_null())
    .group_by('player_id')  # Only group by player_id, not team_id!
    .agg([
        # Sum all granular metric points
        pl.sum('finishing_points').alias('total_finishing'),
        pl.sum('chance_creation_points').alias('total_chance_creation'),
        pl.sum('ball_progression_points').alias('total_ball_progression'),
        pl.sum('dribbling_points').alias('total_dribbling'),
        pl.sum('ball_winning_points').alias('total_ball_winning'),
        pl.sum('defensive_actions_points').alias('total_defensive_actions'),
        pl.sum('passing_accuracy_points').alias('total_passing_accuracy'),
        pl.sum('long_passing_points').alias('total_long_passing'),
        
        # Count events
        pl.len().alias('total_events'),
    ])
)

# Join position and player info
player_granular = (
    player_granular
    .join(player_positions.select(['player_id', 'position']), on='player_id', how='left')
    .join(player_info, on='player_id', how='left')
)

# Fill null player names with placeholder
player_granular = player_granular.with_columns([
    pl.when(pl.col('player_name').is_null())
    .then(pl.concat_str([pl.lit("Player_"), pl.col('player_id')]))
    .otherwise(pl.col('player_name'))
    .alias('player_name')
])

print(f" Fixed null player names")

# Filter for >500 minutes
player_granular = player_granular.filter(pl.col('estimated_total_minutes') >= 500)

print(f" Aggregated metrics for {len(player_granular)} unique players (>500 minutes)")
print(f"   Unique player IDs: {player_granular['player_id'].n_unique()}")

# Verify no duplicates
assert len(player_granular) == player_granular['player_id'].n_unique(), "Still have duplicates!"

# Calculate per-90 for each metric
metric_columns = [
    'finishing', 'chance_creation', 'ball_progression', 'dribbling',
    'ball_winning', 'defensive_actions', 'passing_accuracy', 'long_passing'
]

for metric in metric_columns:
    player_granular = player_granular.with_columns([
        ((pl.col(f'total_{metric}') / pl.col('estimated_total_minutes')) * 90)
        .alias(f'{metric}_per90')
    ])

print("\n Calculated per-90 for all metrics (no duplicates)")

print("PER-90 METRIC DISTRIBUTIONS")

for metric in metric_columns:
    col = f'{metric}_per90'
    mean_val = player_granular[col].mean()
    max_val = player_granular[col].max()
    print(f"{metric:25s}: Mean={mean_val:>6.2f} | Max={max_val:>6.2f}")

# Top players by each metric
print("TOP 5 PLAYERS BY EACH METRIC (Per-90)")

for metric in metric_columns:
    print(f"\n{metric.upper().replace('_', ' ')}:")
    print(player_granular
          .select(['player_name', 'position', f'{metric}_per90', 'matches_played'])
          .sort(f'{metric}_per90', descending=True)
          .head(5))

# Show Amiri specifically
print("\nNadiem Amiri check:")
amiri = player_granular.filter(pl.col('player_name') == 'Nadiem Amiri')
if len(amiri) > 0:
    print(amiri.select(['player_id', 'player_name', 'position', 'matches_played', 'total_events']))
else:
    print("  Not found (might have <500 minutes)")

Aggregating granular metrics by player...
✅ Fixed null player names
✅ Aggregated metrics for 393 unique players (>500 minutes)
   Unique player IDs: 393

✅ Calculated per-90 for all metrics (no duplicates)

PER-90 METRIC DISTRIBUTIONS
finishing                : Mean=  1.48 | Max= 13.83
chance_creation          : Mean=  3.51 | Max= 24.27
ball_progression         : Mean= 16.55 | Max= 76.22
dribbling                : Mean=  3.47 | Max= 10.07
ball_winning             : Mean=  7.94 | Max= 23.56
defensive_actions        : Mean=  7.97 | Max= 36.58
passing_accuracy         : Mean= 17.01 | Max= 77.04
long_passing             : Mean=  3.08 | Max= 22.03

TOP 5 PLAYERS BY EACH METRIC (Per-90)

FINISHING:
shape: (5, 4)
┌─────────────────┬──────────┬─────────────────┬────────────────┐
│ player_name     ┆ position ┆ finishing_per90 ┆ matches_played │
│ ---             ┆ ---      ┆ ---             ┆ ---            │
│ str             ┆ str      ┆ f64             ┆ u32            │
╞═════════════════╪═

In [ ]:
# Cell 6: Scale metrics using StandardScaler (z-scores)
from sklearn.preprocessing import StandardScaler

print("Scaling metrics using StandardScaler (z-scores)...")

# Scale each metric
scaled_metrics = {}

for metric in metric_columns:
    col_name = f'{metric}_per90'
    
    # Get data
    data = player_granular[col_name].to_numpy().reshape(-1, 1)
    
    # Standardize (z-score)
    scaler = StandardScaler()
    z_scores = scaler.fit_transform(data).flatten()
    
    # Convert to 0-100 scale with mean=50, std=15
    # This means: mean player = 50, 1 std above = 65, 2 std above = 80, etc.
    scaled = (z_scores * 15) + 50
    
    # Clip to realistic range (0-100)
    scaled = np.clip(scaled, 0, 100)
    
    scaled_metrics[f'{metric}_rating'] = scaled
    
    print(f"{metric:25s}: Min={scaled.min():.1f}, Max={scaled.max():.1f}, "
          f"Mean={scaled.mean():.1f}, Std={scaled.std():.1f}")

# Add to dataframe
for metric, ratings in scaled_metrics.items():
    player_granular = player_granular.with_columns([
        pl.Series(metric, ratings)
    ])

print("\n All metrics scaled to 0-100 (mean=50, std=15)")

# Show top players by scaled ratings
print("TOP 5 BY SCALED RATINGS")

for metric in metric_columns:
    rating_col = f'{metric}_rating'
    print(f"\n{metric.upper().replace('_', ' ')} RATING:")
    print(player_granular
          .select(['player_name', 'position', rating_col, 'matches_played'])
          .sort(rating_col, descending=True)
          .head(5))

Scaling metrics using StandardScaler (z-scores)...
finishing                : Min=37.0, Max=100.0, Mean=49.6, Std=13.2
chance_creation          : Min=35.7, Max=100.0, Mean=49.7, Std=13.9
ball_progression         : Min=30.0, Max=100.0, Mean=49.9, Std=14.7
dribbling                : Min=25.8, Max=100.0, Mean=50.0, Std=15.0
ball_winning             : Min=27.4, Max=94.4, Mean=50.0, Std=15.0
defensive_actions        : Min=30.7, Max=100.0, Mean=49.9, Std=14.8
passing_accuracy         : Min=31.1, Max=100.0, Mean=49.9, Std=14.5
long_passing             : Min=31.5, Max=100.0, Mean=49.8, Std=14.2

✅ All metrics scaled to 0-100 (mean=50, std=15)

TOP 5 BY SCALED RATINGS

FINISHING RATING:
shape: (5, 4)
┌─────────────────┬──────────┬──────────────────┬────────────────┐
│ player_name     ┆ position ┆ finishing_rating ┆ matches_played │
│ ---             ┆ ---      ┆ ---              ┆ ---            │
│ str             ┆ str      ┆ f64              ┆ u32            │
╞═════════════════╪══════════╪═

In [ ]:
# Cell 7: Combine metrics into role-specific overall ratings
print("Creating role-specific overall ratings...")

# Define role-specific weights for combining metrics
ROLE_WEIGHTS = {
    'forward': {
        'finishing': 0.40,
        'chance_creation': 0.25,
        'dribbling': 0.20,
        'ball_progression': 0.10,
        'ball_winning': 0.05,
        'defensive_actions': 0.00,
        'passing_accuracy': 0.00,
        'long_passing': 0.00,
    },
    'midfielder': {
        'finishing': 0.10,
        'chance_creation': 0.20,
        'dribbling': 0.10,
        'ball_progression': 0.30,
        'ball_winning': 0.15,
        'defensive_actions': 0.05,
        'passing_accuracy': 0.10,
        'long_passing': 0.00,
    },
    'defender': {
        'finishing': 0.00,
        'chance_creation': 0.00,
        'dribbling': 0.00,
        'ball_progression': 0.15,
        'ball_winning': 0.35,
        'defensive_actions': 0.30,
        'passing_accuracy': 0.10,
        'long_passing': 0.10,
    },
}

print("Role-specific weights:")
for role, weights in ROLE_WEIGHTS.items():
    print(f"\n{role.upper()}:")
    for metric, weight in weights.items():
        if weight > 0:
            print(f"  {metric:25s}: {weight:>5.0%}")

# Calculate overall rating for each player
def calculate_role_overall(row):
    position = row['position']
    if position not in ROLE_WEIGHTS:
        position = 'midfielder'  # Default
    
    weights = ROLE_WEIGHTS[position]
    overall = 0.0
    
    for metric, weight in weights.items():
        if weight > 0:
            overall += row[f'{metric}_rating'] * weight
    
    return overall

overall_ratings = []
for row in player_granular.iter_rows(named=True):
    overall_ratings.append(calculate_role_overall(row))

player_granular = player_granular.with_columns([
    pl.Series('role_overall_rating', overall_ratings)
])

print("\n Role-specific overall ratings calculated")

# Distribution
print(f"\nOverall rating distribution:")
print(f"  Min: {player_granular['role_overall_rating'].min():.1f}")
print(f"  Max: {player_granular['role_overall_rating'].max():.1f}")
print(f"  Mean: {player_granular['role_overall_rating'].mean():.1f}")
print(f"  Std: {player_granular['role_overall_rating'].std():.1f}")

print("\n" + "="*70)
print("TOP 20 PLAYERS - ROLE-SPECIFIC OVERALL RATING")
print("="*70)
print(player_granular
      .select(['player_name', 'position', 'role_overall_rating', 
               'finishing_rating', 'chance_creation_rating', 
               'ball_progression_rating', 'ball_winning_rating',
               'matches_played'])
      .sort('role_overall_rating', descending=True)
      .head(20))

Creating role-specific overall ratings...
Role-specific weights:

FORWARD:
  finishing                :   40%
  chance_creation          :   25%
  dribbling                :   20%
  ball_progression         :   10%
  ball_winning             :    5%

MIDFIELDER:
  finishing                :   10%
  chance_creation          :   20%
  dribbling                :   10%
  ball_progression         :   30%
  ball_winning             :   15%
  defensive_actions        :    5%
  passing_accuracy         :   10%

DEFENDER:
  ball_progression         :   15%
  ball_winning             :   35%
  defensive_actions        :   30%
  passing_accuracy         :   10%
  long_passing             :   10%

✅ Role-specific overall ratings calculated

Overall rating distribution:
  Min: 34.8
  Max: 98.0
  Mean: 53.8
  Std: 11.8

TOP 20 PLAYERS - ROLE-SPECIFIC OVERALL RATING
shape: (20, 8)
┌───────────────┬──────────┬───────────────┬───────────────┬──────────────┬──────────────┬──────────────┬──────────────┐


In [ ]:
# Cell 8: Create leaderboards with event breakdowns
print("Creating detailed leaderboards with event counts...")

# Check if event counts already exist
if 'goals' not in player_granular.columns:
    # Count key events per player
    event_counts = (
        all_events
        .filter(pl.col('player_id').is_not_null())
        .group_by('player_id')
        .agg([
            # Goals
            pl.col('event_type').filter(
                (pl.col('event_type') == 'SHOT') & (pl.col('result') == 'GOAL')
            ).count().alias('goals'),
            
            # Shots
            pl.col('event_type').filter(pl.col('event_type') == 'SHOT').count().alias('shots'),
            
            # Assists
            pl.col('pass_type').filter(pl.col('pass_type') == 'SHOT_ASSIST').count().alias('assists'),
            
            # Progressive passes
            pl.col('ball_progression_points').filter(
                pl.col('ball_progression_points') >= 2.0
            ).count().alias('progressive_passes'),
            
            # Tackles/Interceptions
            pl.col('event_type').filter(
                pl.col('event_type').is_in(['DUEL', 'INTERCEPTION'])
            ).count().alias('defensive_actions'),
            
            # Successful passes
            pl.col('event_type').filter(
                (pl.col('event_type') == 'PASS') & (pl.col('result') == 'COMPLETE')
            ).count().alias('passes_completed'),
        ])
    )
    
    # Join with main dataframe
    player_granular = player_granular.join(event_counts, on='player_id', how='left')
    print(" Event counts added")
else:
    print(" Event counts already exist, skipping...")

# Top 10 Finishers
print("\n" + "="*70)
print("TOP 10 FINISHERS (Goals + Shooting)")
print("="*70)
print(player_granular
      .filter(pl.col('position') == 'forward')
      .select(['player_name', 'finishing_rating', 'goals', 'shots', 'matches_played'])
      .sort('finishing_rating', descending=True)
      .head(10))

# Top 10 Playmakers
print("TOP 10 PLAYMAKERS (Assists + Chance Creation)")
print(player_granular
      .select(['player_name', 'position', 'chance_creation_rating', 
               'assists', 'matches_played'])
      .sort('chance_creation_rating', descending=True)
      .head(10))

# Top 10 Ball Progressors
print("TOP 10 BALL PROGRESSORS (Progressive Passes/Carries)")
print(player_granular
      .select(['player_name', 'position', 'ball_progression_rating',
               'progressive_passes', 'matches_played'])
      .sort('ball_progression_rating', descending=True)
      .head(10))

# Top 10 Ball Winners - FIXED COLUMN NAME
print("TOP 10 BALL WINNERS (Tackles + Interceptions)")
print(player_granular
      .filter(pl.col('position').is_in(['defender', 'midfielder']))
      .select(['player_name', 'position', 'ball_winning_rating',
               'defensive_actions', 'matches_played'])  # ← Changed here
      .sort('ball_winning_rating', descending=True)
      .head(10))

# Top 10 Overall by Position
print("TOP 10 BY POSITION (Role-Specific Overall)")

for position in ['forward', 'midfielder', 'defender']:
    print(f"\n{position.upper()}S:")
    print(player_granular
          .filter(pl.col('position') == position)
          .select(['player_name', 'role_overall_rating',
                   'goals', 'assists', 'progressive_passes', 'defensive_actions',  # ← Changed here
                   'matches_played'])
          .sort('role_overall_rating', descending=True)
          .head(10))

print("\n All leaderboards created!")

Creating detailed leaderboards with event counts...
✅ Event counts already exist, skipping...

TOP 10 FINISHERS (Goals + Shooting)
shape: (10, 5)
┌───────────────────┬──────────────────┬───────┬───────┬────────────────┐
│ player_name       ┆ finishing_rating ┆ goals ┆ shots ┆ matches_played │
│ ---               ┆ ---              ┆ ---   ┆ ---   ┆ ---            │
│ str               ┆ f64              ┆ u32   ┆ u32   ┆ u32            │
╞═══════════════════╪══════════════════╪═══════╪═══════╪════════════════╡
│ Loïs Openda       ┆ 100.0            ┆ 24    ┆ 111   ┆ 34             │
│ Victor Boniface   ┆ 100.0            ┆ 14    ┆ 78    ┆ 23             │
│ Harry Kane        ┆ 100.0            ┆ 36    ┆ 141   ┆ 32             │
│ Player_44929      ┆ 100.0            ┆ 18    ┆ 99    ┆ 30             │
│ Serhou Guirassy   ┆ 100.0            ┆ 28    ┆ 93    ┆ 28             │
│ Donyell Malen     ┆ 98.65236         ┆ 13    ┆ 74    ┆ 27             │
│ Andrej Kramaric   ┆ 94.068212        ┆

In [ ]:
# Cell 9: Save role-specific ratings
print("Saving role-specific player ratings...")

# Save to parquet and CSV
player_granular.write_parquet(PROCESSED_DIR / "player_ratings_role_specific.parquet")
player_granular.write_csv(PROCESSED_DIR / "player_ratings_role_specific.csv")

print(f" Saved to:")
print(f"   {PROCESSED_DIR / 'player_ratings_role_specific.parquet'}")
print(f"   {PROCESSED_DIR / 'player_ratings_role_specific.csv'}")

# Summary statistics
print("ROLE-SPECIFIC RATINGS SYSTEM COMPLETE! ")

print(f"\nDataset Summary:")
print(f"  Total players: {len(player_granular)}")
print(f"  Forwards: {(player_granular['position'] == 'forward').sum()}")
print(f"  Midfielders: {(player_granular['position'] == 'midfielder').sum()}")
print(f"  Defenders: {(player_granular['position'] == 'defender').sum()}")

print(f"\nGranular Metrics: {len(metric_columns)}")
for i, metric in enumerate(metric_columns, 1):
    print(f"  {i}. {metric}")

print(f"\nRating Statistics:")
print(f"  Overall rating range: {player_granular['role_overall_rating'].min():.1f} - {player_granular['role_overall_rating'].max():.1f}")
print(f"  Overall rating mean: {player_granular['role_overall_rating'].mean():.1f}")
print(f"  Overall rating median: {player_granular['role_overall_rating'].median():.1f}")



In [ ]:
# Cell 10: Compare both approaches
print("COMPARING BOTH APPROACHES")
# Load old ratings
old_ratings = pl.read_parquet(PROCESSED_DIR / "player_ratings_final.parquet")

# Compare top 20 overall
print("\nTOP 20 - APPROACH A (Simple Event-Based):")
print(old_ratings
      .select(['player_name', 'position', 'overall_rating'])
      .sort('overall_rating', descending=True)
      .head(20))

print("\nTOP 20 - APPROACH B (Role-Specific Granular):")
print(player_granular
      .select(['player_name', 'position', 'role_overall_rating'])
      .sort('role_overall_rating', descending=True)
      .head(20))

# Position distribution in top 20
print("\nPosition Distribution in Top 20:")
print("Approach A:")
print(old_ratings.sort('overall_rating', descending=True).head(20)['position'].value_counts())
print("\nApproach B:")
print(player_granular.sort('role_overall_rating', descending=True).head(20)['position'].value_counts()) 

COMPARING BOTH APPROACHES

TOP 20 - APPROACH A (Simple Event-Based):
shape: (20, 3)
┌────────────────────┬────────────┬────────────────┐
│ player_name        ┆ position   ┆ overall_rating │
│ ---                ┆ ---        ┆ ---            │
│ str                ┆ str        ┆ f64            │
╞════════════════════╪════════════╪════════════════╡
│ Min-jae Kim        ┆ defender   ┆ 83.375663      │
│ Granit Xhaka       ┆ midfielder ┆ 83.349088      │
│ Nico Schlotterbeck ┆ defender   ┆ 80.281326      │
│ Timo Hübers        ┆ defender   ┆ 77.703839      │
│ Willian Pacho      ┆ defender   ┆ 76.791501      │
│ …                  ┆ …          ┆ …              │
│ Philipp Lienhart   ┆ defender   ┆ 68.601409      │
│ Edmond Tapsoba     ┆ midfielder ┆ 67.987629      │
│ Hiroki Ito         ┆ defender   ┆ 67.224339      │
│ Jeff Chabot        ┆ defender   ┆ 66.933372      │
│ Marco Friedl       ┆ defender   ┆ 66.663863      │
└────────────────────┴────────────┴────────────────┘

TOP 20 - APPRO